In [1]:
from yaml import load, Loader
with open("config.yaml", "r") as f:
    config = load(f, Loader=Loader)

In [2]:
import requests

API_KEY = config['API_KEY']
BASE_URL = "https://trefle.io/api/v1/plants"


# for page in range(1, total_pages + 1):
params = {"token": API_KEY, "page": 1, "per_page": 10}
response = requests.get(BASE_URL, params=params)
response.json()

{'data': [{'id': 77116,
   'common_name': 'Evergreen oak',
   'slug': 'quercus-rotundifolia',
   'scientific_name': 'Quercus rotundifolia',
   'year': 1785,
   'bibliography': 'Encycl. 1: 723 (1785)',
   'author': 'Lam.',
   'status': 'accepted',
   'rank': 'species',
   'family_common_name': None,
   'genus_id': 3519,
   'image_url': 'https://d2seqvvyy3b8p2.cloudfront.net/40ab8e7cdddbe3e78a581b84efa4e893.jpg',
   'synonyms': ['Quercus ilex var. oleoides',
    'Quercus ilex subvar. rotundifolia',
    'Quercus ilex f. macrophylla',
    'Quercus ilex f. oleoides',
    'Quercus ilex var. calicina',
    'Quercus ilex subsp. rotundifolia',
    'Quercus lyauteyi',
    'Quercus ballota var. rotundifolia',
    'Quercus ilex f. brevicupulata',
    'Quercus ilex subvar. major',
    'Quercus ilex var. pendula',
    'Quercus rotundifolia f. dolichocalyx',
    'Quercus calicina',
    'Quercus rotundifolia f. pilosella',
    'Quercus rotundifolia f. macrocarpa',
    'Quercus ilex var. rotundifolia',

In [3]:
response.json()['data'][0]

{'id': 77116,
 'common_name': 'Evergreen oak',
 'slug': 'quercus-rotundifolia',
 'scientific_name': 'Quercus rotundifolia',
 'year': 1785,
 'bibliography': 'Encycl. 1: 723 (1785)',
 'author': 'Lam.',
 'status': 'accepted',
 'rank': 'species',
 'family_common_name': None,
 'genus_id': 3519,
 'image_url': 'https://d2seqvvyy3b8p2.cloudfront.net/40ab8e7cdddbe3e78a581b84efa4e893.jpg',
 'synonyms': ['Quercus ilex var. oleoides',
  'Quercus ilex subvar. rotundifolia',
  'Quercus ilex f. macrophylla',
  'Quercus ilex f. oleoides',
  'Quercus ilex var. calicina',
  'Quercus ilex subsp. rotundifolia',
  'Quercus lyauteyi',
  'Quercus ballota var. rotundifolia',
  'Quercus ilex f. brevicupulata',
  'Quercus ilex subvar. major',
  'Quercus ilex var. pendula',
  'Quercus rotundifolia f. dolichocalyx',
  'Quercus calicina',
  'Quercus rotundifolia f. pilosella',
  'Quercus rotundifolia f. macrocarpa',
  'Quercus ilex var. rotundifolia',
  'Quercus sugaro',
  'Quercus ilex subvar. pendula',
  'Quercu

In [4]:
from sqlmodel.ext.asyncio.session import AsyncSession
from sqlalchemy.ext.asyncio import create_async_engine, async_sessionmaker


engine = create_async_engine(config['sqlalchemy_url'])
session_maker = async_sessionmaker(
    engine, expire_on_commit=False, class_=AsyncSession)

In [5]:

from sqlmodel import SQLModel, Field
from uuid import UUID, uuid4
from datetime import date


class Plant(SQLModel, table=True):
    __tablename__ = "plants"
    __table_args__ = {'extend_existing': True}
    id: UUID = Field(primary_key=True, default_factory=uuid4)
    name: str
    bibliography: str
    slug: str
    year: date
    image_url: str
    family: str
    genus: str
    rank: str

In [6]:
from datetime import date

date(year=1997, month=1, day=1)

datetime.date(1997, 1, 1)

In [7]:
from datetime import date
async with session_maker() as session:
    async with session.begin():
        for plant in response.json()['data']:
            session.add(Plant(name=plant['common_name'],
                              bibliography=plant['bibliography'],
                              slug=plant['slug'],
                              year=date(year=plant['year'], month=1, day=1),
                              image_url=plant['image_url'],
                              family=plant['family'],
                              genus=plant['genus'],
                              rank=plant['rank'],
                              ))